<a href="https://colab.research.google.com/github/glo-code/ecommerce-analysis/blob/main/ecommerce_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import datetime

In [ ]:
df=pd.read_csv('Amazon Sale Report.csv',parse_dates=True,on_bad_lines='skip',low_memory=False)
df



```
Indagine sui clienti


```



Ordine ID e Status

In [ ]:
data=pd.to_datetime(df['Date'])
df['months']=data.dt.month_name()
df['year']=data.dt.year
df['day']=data.dt.day


group=df.groupby(['day','months','year'])[['Order ID','Status']].value_counts().reset_index()
mask=np.where(group['count']<group['count'].max())[0]
print('Order less than max:',group['count'].iloc[mask].sum(),'in the day:',group.iloc[mask])
mask2=group['count'][group['count']==group['count'].max()]
'Order == max:',mask2.value_counts(),'in the day:',group.iloc[mask2.index]


Notiamo che il massimo di ordini da parte di un cliente è di 12 mentre il minimo è 1.Il massimo di ordini si è verificato sia ad aprile che a maggio, soltanto che a maggio si è trattato non di ordini spediti ma cancellati dallo stesso cliente.
Abbiamo invece contro i 2 ordini più alti, 91354 ordini minimi da più clienti.


In [ ]:
count=group.groupby('months')['Status'].value_counts().unstack()
count=count.fillna(0).astype(int)
count

In [ ]:
perc_precise=count/count.sum()*100
perc_precise.round(2).reset_index()


labels=set(df['Status'])

conteggio=0
for categorie in perc_precise:
    categorie=perc_precise.iloc[conteggio]
    print(categorie.name)
    ax,fig=plt.subplots()
    plt.pie(categorie,labels=labels,autopct='%1.1f%%')
    conteggio=conteggio+1
    if conteggio==len(perc_precise.index):
       break


In [ ]:
lista_mesi=['June','May','April','March']
ordini_totali=[]
for mese in lista_mesi:
    result=group['count'][group['months']==mese].sum()
    ordini_totali.append(result)

ordini_totali=pd.Series(ordini_totali)
print('Ordini per mese',ordini_totali)
print('Ordini totali',ordini_totali.sum())
perc=ordini_totali/ordini_totali.sum()*100
perc=perc.round(2)

ax,fig=plt.subplots()
plt.pie(perc,labels=lista_mesi,autopct='%1.1f%%')
plt.show()



Notiamo che ad Aprile abbiamo il maggior numero di ordini spediti (17.8%) e cancellati (15,5%). Abbiamo anche un modesto numero di ordini ritornati al venditore e in sospeso(16%). A giugno invece alto numero di ordini spediti ma anche cancellati anche se sono di meno rispetto ad aprile. Pochi però i resi e molti sospesi. A marzo nessun ordine sospeso e pochi cancellati ma ben il 44% degli ordini persi nel transito e 27% cancellati rispetto al 15,5% di aprile. E' il mese in cui si hanno avuti meno ordini. A maggio la maggior parte spediti e cancellati ed un numero moderato minore rispetto ad aprile di resi (21.6% rispetto al 16% di aprile). Pochi sospesi (18% rispetto ad aprile) e il 17% di persi nel transito e cancellati.

Quindi si è venduto di più tra aprile e maggio (47% e 40%) e modertamente a giugno (12%) e di meno a marzo (0.2%).



In [ ]:
total=df['Status'].value_counts().sum()
status=df['Status'].value_counts().apply(lambda x:x/total*100).round(2).reset_index()
#status=status[status['count']!=0.0]
value=status['count']
fig,ax=plt.subplots()
labels=status['Status']
plt.pie(value,labels=labels,autopct='%1.1f%%')#,startangle=90,shadow=True,labeldistance=0.80,pctdistance=0.5)
status

Notiamo che la maggior parte degli ordini nei mesi indicati, sono stati spediti (circa il 58,8%) mentre il 25 % è stato spedito e consegnato al compratore. Il 14% invece è stato cancellato mentre soltanto 1.8% degli ordini è stato mandato indietro.
Soltanto per il 0,2% invece è stato effettuato il reso ma ancora l'ordine non è ritornato al venditore.



In [ ]:

sns.catplot(group,x=group['months'],y=group['count'],kind='bar',hue='Status')
plt.show()
group

Tipo di prodotto acquistato


In [ ]:
clothes=df['Category'].value_counts().reset_index()
perc_vestiti=clothes['count']/clothes['count'].sum()*100
vestiti_mesi=df.groupby('Category')['months'].value_counts().unstack()
vestiti_mesi.fillna(0).astype(int)
clothes.sum()

In [ ]:
sns.barplot(clothes,x='Category',y='count')
plt.xticks(rotation=90)
plt.show()

Notiamo che i prodotti più acquistati in assoluto sono i set e la kurta e moderatamente il western dress e top. I prodotti meno acquistati sono ethnic dress, blouse, bottom e saree e ancora meno (soltanto 3 acquisti) il dupatta.
Dalle percentuali notiamo che il 39% acquista il set e il 38% il kurta mentre il 12% il western dress e soltanto l'8.2% il top.

In [ ]:
ax,fig=plt.subplots()
plt.pie(perc_vestiti,labels=clothes['Category'],autopct='%1.1f%%')
plt.show()
perc_vestiti

In [ ]:
#ax,fig=plt.subplots()
vestiti_mesi.plot(kind='bar')
plt.show()

I set e i kurta sono stati più venduti nel mese di aprile, di più i set e i set anche nel mese di maggio sono stati più venduti rispetto ai kurta, che invece sono stati più venduti durante il mese di giugno. I top e i western dress invece hanno avuto poche vendite rispetto ai set e ai kurta ma i western dress nel mese di maggio e giugno sono stati più venduti rispetto ai top.

In [ ]:

taglie=pd.crosstab(df['Category'],[df['Size']]).reset_index()
taglie.round(4)

In [ ]:
sns.barplot(taglie)

La taglia più venduta è la M/L mentre quella meno venduta è la XS e la 3XL. Mentre le taglie oltre la 3XL sono vendute ancora più raramente e anche la taglia Free.

Spesa


In [ ]:
'Il prezzo massimo/minimo e medio sono:',df['Amount'].describe()[['max','min','mean']]
prezzo=df.groupby('months')['Amount'].mean().reset_index()
prezzo

In [ ]:
c=df.groupby('Category')['Amount'].value_counts().unstack()
c.fillna(0).astype(int)

In media i clienti spendono 648,56 dollari per acquistare la merce in totale. Il prezzo medio ad Marzo è stato 627, a Aprile 626 per poi aumentare a Maggio 663 e scendere leggermente a Giugno 661. Quindi in teoria i prezzi sono saliti tra Maggio e Giugno.

Notiamo però che la maggior parte degli acquisti di alcuni articoli sono costati 0.00 e questa è una cosa strana. Anche se notiamo che tra i kurta ben 44 siano stati venduti a 249.00 mentre 14 bottom a 229.00 e 7 blouse a 249.00 ma il resto è minore.
Perchè la maggior parte degli articoli sono stati acquistati gratis?

In [ ]:
promozioni=df.groupby('promotion-ids')['Amount'].value_counts().reset_index()
vestiti_promo=df.groupby('promotion-ids')[['Category','months']].value_counts()#.reset_index()#'amount' con category
vestiti_promo_2=vestiti_promo.copy().reset_index()
print(promozioni[promozioni['count']==promozioni['count'].max()])
promo=df['promotion-ids'].value_counts()#.reset_index()
#kurta=vestiti_promo[vestiti_promo['Category']=='kurta']
#kurta[kurta['Amount']==0.0].sum()
set(promozioni[promozioni['Amount']==0.0]['promotion-ids'].value_counts().reset_index()['promotion-ids'])
#set(vestiti_promo['Sales Channel '])
#perc_promo=promo['count']/promo['count'].sum()*100
#promo_1=promo.copy()
#promo_1['count']=perc_promo.round(2)
#promo_1['count'][(promo_1['count']<57.75)&(promo_1['count']>0.0)].sum()

#print(promo.sum())


vestiti_promo.unstack().fillna(0).astype(int)




La promozione più utilizzata è VPC-44571-44201853, usata per circa 46100 volte,quindi si tratta di spedizione gratuita (Il numero più alto di articoli acquistati con questa promozione (1754) è costato 399.0 ed in particolare 1687 sono i kurta acquistati 49877 volte (3,38% degli acquisti)).
458 persone hanno invece utilizzato la promozione collegata alle carte platino, attraverso la quale amazon applica lo sconto sull'ordine permettendo però al venditore di ricevere il guadagno completo.
Questa promozione invece è stata utilizzata per comprare i kurta a 0.0 (291 quindi 0.58%) ma è strano perchè dovrebbe trattarsi di una promozione applicabile solo ai vendito
ri.
In generale 781 articoli sono stati venduti per 0.0 (128975 articolo in totale quindi 0,61%) ed è dovuto a tre promozioni sia questa e due coupon che dovrebbero abbassare il prezzo ma non rendere l'articolo gratuito. Sono tutte promozioni provenienti da Amazon.
In totale il 52% degli articoli sono stati comprati tramite la promozione IN Core Free Shipping 2015/04/08 23-48-5-108
come avevamo già notato mentre lo 0,57% con la promozione free-fancing ed anche sullo 0 è la percentuale dei coupon duplicates.


In [ ]:
mask=vestiti_promo_2['promotion-ids'].str.contains('IN Core')
in_core=vestiti_promo_2[['promotion-ids','count','Category','months']][mask==True]
coupon=vestiti_promo_2[['promotion-ids','count','Category','months']][mask==False]
coupon=coupon[['promotion-ids','count','Category','months']][coupon['promotion-ids'].str.contains('Amazon')==False]

incore=in_core.groupby(['Category','months'])[['count','promotion-ids']].sum()

coupon=coupon.groupby(['Category','promotion-ids','months'])[['count']].sum()

amazon=vestiti_promo_2['promotion-ids'].str.contains('Amazon')
amazon_1=vestiti_promo_2[['promotion-ids','count','Category','months']][amazon==True]
amazon_2=amazon_1.groupby(['Category','months'])[['count','promotion-ids']].sum()





sns.catplot(incore,kind='bar',x='Category',y='count')
plt.xticks(rotation=90)
plt.title('Clothes bought by IN Core promotion in April')
plt.show()
sns.catplot(amazon_2,kind='bar',x='Category',y='count')
plt.title('Clothes bought by Amazon promotion in April')
plt.xticks(rotation=90)
plt.show()
sns.catplot(coupon,kind='bar',x='Category',y='count',hue='promotion-ids',aspect=2,height=10)
plt.xticks(rotation=90)
plt.title('Clothes bought by other promotion in April')
plt.show()

coupon



In aprile notiamo che le promozioni più usate per comprare (Western Dress) sono le promozioni duplicated. La promozione VCP invece è stata usata moderatamente per i western dress (spedizione gratuita) ma poco per quanto riguarda gli altri articoli come anche duplicated.
Quella di Amazon di più su kurta e set e lo stesso la promozioni IN CORE.


Perchè abbiamo quegli articoli venduti a 0.0?

In [ ]:
print(df['Amount'][df['Amount']==0.0].value_counts())
articoli=df.groupby('Amount')[['months','year','Order ID','Status','promotion-ids','Courier Status']].value_counts().reset_index()
nulli=articoli[['Amount','months','year','Status','Order ID','promotion-ids','Courier Status']][articoli['Amount']==0.0]
count_ordini=nulli['months'].value_counts()
print(count_ordini)
lista_guadagno=[]
for months in ['April','May','June']:
    guadagno=df[['Amount','months']][df['months']==months]['Amount'].sum()
    income=[months,guadagno]
    lista_guadagno.append(income)


df_guadagni=pd.DataFrame(lista_guadagno)
df_guadagni

print(nulli['promotion-ids'].value_counts().reset_index()[nulli['promotion-ids'].value_counts().reset_index()['promotion-ids'].str.contains('VPC')]['count'].sum())

nulli['Order ID'].duplicated().value_counts() # non ci sono ordini gratis fatti dallo stesso cliente
stato=nulli['Status'].value_counts()
stato/stato.sum()*100
stato.sum()

nulli['Courier Status'].value_counts()

df['Amount'][df['Amount']==0.0]


Notiamo che nessuno di questi ordini senza pagamento è stato cancellato e che non ci sono ordini gratis dallo stesso cliente. Questi ordini gratis si sono avuti nel numero massimo ad Aprile mentre moderatamente a Giugno e poco meno a Maggio.
Potrebbero anche essere valori anomali.Comunque ad Aprile si è guadagnato di più mentre a Maggio moderatamente e a Giugno ancora meno. Questi potrebbero essere il motivo di questo calo di fatturato tra Aprile e Giugno?
Notiamo che la maggior parte di questi ordini è stato consegnato al cliente (87%) mentre 50 sono stati resi (6%), 5 sospesi (0.64%), 2 persi nel transito (0.25%) e 2 sono in fase di reso (in totale 57 di questi ordini sono nulli per questo motivo). Nessuno di loro si tratta di ordini cancellati ma 13 sono non spediti
Invece la maggior parte ha utilizzato la promozione Amazon (779) mentre soltanto 2 VCP coupon.
- 57 ordini nulli per reso,sospeso,perso nel transito e in fase di reso su 781 + 13 non spediti (in tutto 8,96%)
- 779 comprati con la promozione Amazon (non sappiamo cosa comporti) su 781 99,74%
- La maggior parte di loro si è avuta nel mese di Aprile
- Potrebbe anche essere un errore, in quanto abbiamo 7795 valori nulli su 128975 incassi 6,54%
- Sono 781 ordini nulli su 128975 ordini in totale (0,61%)


In [ ]:


sns.catplot(data=count_ordini,kind='bar')
plt.title('Ordini nulli nei diversi mesi')
plt.show()





Valori anomali e nulli sulla colonna Amount?

In [ ]:
df['Amount'].describe()
new_df=df.copy()
#new_df['Amount']=new_df['Amount'][new_df['Amount']!=0.0]
new_df['Amount']

In [ ]:
df['Amount'][df['Amount'].isnull()==True]
#new_df['Amount'].fillna(new_df['Amount'].median())
new_df['Amount'].describe()

In [ ]:
#metodo percentili
sup=df['Amount'].quantile(0.99)
inf=df['Amount'].quantile(0.01)
out=df['Amount'].loc[(df['Amount']>sup)&(df['Amount']<inf)]
out

In [ ]:
#metodo zscore
superiore=df['Amount']+3*df['Amount'].std()
inferiore=df['Amount']-3*df['Amount'].std()
outliers=df['Amount'].loc[(df['Amount']>superiore)&(df['Amount']<inferiore)]
print(outliers)

In [ ]:
index=np.where(new_df['Amount']==0.0)[0]
new_df['free_order']=new_df['Amount'].iloc[index]
new_df['free_order']=new_df['free_order'].fillna('Order not null')
new_df['free_order']=new_df['free_order'].apply(lambda f:f if f!=0.0 else 'Order null')
count=new_df['free_order'].iloc[index].value_counts().sum()

#new_df['Amount']=new_df['Amount'][new_df['Amount']!=0.0]
print(new_df['free_order'],count,len(df['Amount']),len(new_df['Amount']))

In [ ]:
#metodo interquartile
q1=df['Amount'].quantile(0.25)
q3=df['Amount'].quantile(0.75)
irq=q3-q1
upper=q3+(1.5*irq)
lower=q1-(1.5*irq)
new_df=df.loc[(df['Amount']<upper)&(df['Amount']>lower)]
indice=np.array(df['Amount'][(df['Amount']>upper)&(df['Amount']>lower)].reset_index()['index'])
num=len(indice)
out=df[['Category','months']].iloc[indice].value_counts()
out1=df['months'].iloc[indice].value_counts()
out2=df['Order ID'].iloc[indice].value_counts()
print(out,out1,out2)
new_df=df.loc[(df['Amount']<upper)&(df['Amount']>lower)]
print('Valori tolti al di sopra del limite superiore',indice)
print('Numero valori anomali',num,'Lunghezza nuova colonna',len(df['Amount'])-len(new_df['Amount']))


fig,ax=plt.subplots(2,1,figsize=(8,9))
ax[1].boxplot(new_df['Amount'])
ax[0].boxplot(df['Amount'])
plt.plot()


fig,ax=plt.subplots(2,1,figsize=(8,9))
ax[1].hist(new_df['Amount'],bins=50)
ax[1].set_title('After remove outliers')
ax[0].hist(df['Amount'],bins=50)
ax[0].set_title('Before remove outliers')
plt.show()
print('Before',df['Amount'].describe())
print('After',new_df['Amount'].describe())
print('Asimmetria',(df['Amount'].skew(),new_df['Amount'].skew()),'Curtosi',(df['Amount'].kurt(),new_df['Amount'].kurt()))

In [ ]:
guadagno=new_df.groupby('months')['Amount'].sum().reset_index()
guadagno_prima=df.groupby('months')['Amount'].sum().reset_index()
print(guadagno)
print(guadagno_prima)
somma2=new_df['Amount'].sum()
somma1=df['Amount'].sum()
print(somma1,somma2)



- I valori anomali abbassano il guadagno mentre gli ordini gratis non hanno un uguale impatto, anzi lasciano il guadagno invariato (vedremo poi la correlazione).

- Il numero dei valori anomali identificati dall'intervallo interquartile è 3600 su 117580 (3,06%).
Dall'istogramma si nota chiaramente che con i valori anomali l'intervallo dei dati era compreso tra 0 e 16000 (i valori erano maggiormente compresi prima di 1000 e calavano dopo 1000 fino a scomparire prima di 2000 (max 5584)) mentre senza è un intervallo più piccolo tra 0 e 5000 (maggiormente compresi tra da metà 200 a 800 per poi calare fino a 1200 (max 1299)).
La distribuzione in entrambi casi è poco asimmetrica ma nel primo caso la curtosi, quindi la concentrazione dei dati intorno alla media era maggiore mentre nel secondo caso questa togliendo i valori anomali è stata ridotta molto.
I dati sembrano distribuiti più ampiamente.


- Si è venduto di meno a marzo, picco basso in assoluto, e giugno quindi ai cambiamenti di stagione.

- I valori anomali si concentrano sul guadagno relativo al set (3487) mentre 50 su Western Dress, 34 kurta e 19 Ethic dress e questo vuol dire che i prezzi del set sono forse la maggiore causa dei valori anomali e il fatturato ha presentato maggiori valori anomali tra maggio e giugno e moderatamente ad aprile.
Il più alto numero di ordini con valori anomali si è avuto a Giugno dallo stesso cliente (4) ed infatti a Giugno si è avuto un calo delle vendite.
Anche a Maggio calo delle vendite ed è il mese in i set con valori anomali sono stati più acquistati.

In [ ]:
mes=['March','April','May','June']
sns.relplot(data=new_df,x='months',y='Amount',kind='line')
plt.show()

Shop da cui i clienti acquistano

In [ ]:

channel=new_df[['Fulfilment','fulfilled-by','Sales Channel ','ship-service-level']].apply(lambda x:x.value_counts()).fillna(0).astype(int)
perc_channel=channel/channel.sum()*100
perc_channel.round(1)


In [ ]:
new_df[['ship-service-level','fulfilled-by']][(new_df['ship-service-level']=='Standard')|(new_df['fulfilled-by'].isin(['Easy Ship','Merchant']))].value_counts()

- I clienti hanno comprato su Amazon (117580) e la maggior parte ha beneficiato della spedizione espressa (68%,80167) rispetto a quella standard (31%,915).

- I fornitori principali sono: al primo posto Amazon seguito da Merchant e Easy Ship (in ugual numero) (69% per Amazon e 31% per gli altri due).

- Il tipo di spedizione dipende anche dal tipo di fornitore.
La spedizione standard invece è stata utlizzata di più su Merchant che su Amazon (Merchant è uno strumento per far comparire la propria merce sulla barra di ricerca google da cui i clienti possono acquistare più facilmente e che promuove
la merce).
Lo stesso numero di persone ha utilizzato quella standard su Easy Ship.
Quindi Amazon ha il primato sulla spedizione espressa.


In [ ]:

provenienza=new_df['ship-city'].value_counts().reset_index()
stato=new_df['ship-state'].value_counts().reset_index()
stato_max=stato.iloc[stato['count'].nlargest(11).index].reset_index()
provenienza_max=provenienza.apply(lambda max:max.sort_values(ascending=True))
primi_dieci=provenienza_max.iloc[0:11]
perc_city=primi_dieci['count']/provenienza['count'].sum()*100
perc_city_1=[c for c in primi_dieci['ship-city'].to_list()]+[''.join(str(numb))+'%' for numb in perc_city.round(2).to_list()]
perc_stato=stato_max['count']/stato['count'].sum()*100
perc_stato_1=[s for s in stato_max['ship-state']]+[''.join(str(state))+'%' for state in perc_stato.round(2).to_list()]

fig,ax=plt.subplots(2,1,figsize=(8,15))
ax[0].bar(primi_dieci['ship-city'],primi_dieci['count'])
ax[0].tick_params(axis='x',labelrotation=90)
#ax[0].set_xticklabels(labels=perc_city_1)
ax[0].set_title('Top Ten city')
ax[1].bar(stato_max['ship-state'],stato_max['count'])
ax[1].set_title('Top Ten state')
ax[1].tick_params(axis='x',labelrotation=90)
#ax[1].set_xticklabels(labels=perc_stato_1)
plt.show()

print(perc_city_1)
perc_stato_1


- Il più alto numero di clienti proviene dalla città di Bengaluru e lo stato di Maharashtra e tutti i clienti sono provenienti dalla regione IN.

Provenienza clienti

In [ ]:
new_df['ship-country'].value_counts()

Quantità di vestiti acquistata e frequenza di acquisto

In [ ]:
print(new_df['Qty'].astype(int).value_counts())
qty=new_df.groupby('Qty')[['Order ID','Category','months']].value_counts().reset_index()
gratis=qty['Qty'][qty['Qty']==0.0].index
clothes=new_df['Category'].unique()
for m,n in zip(mes,clothes):
    lista=n,qty.iloc[gratis][qty['Category']==n]['count'].sum()
    lista2=n,qty[qty['Category']==n]['count'].sum()
    print(lista,lista2)
qty1=qty.groupby('Qty')[['months','Category']].value_counts()
print(new_df['Qty'].mean())
qty2=new_df.groupby('Qty')['promotion-ids'].value_counts().reset_index()
qty2['Qty'].unique()
qty3=new_df.groupby('Qty')[['Category','Courier Status','months']].value_counts().reset_index()
unshipped=qty3[qty3['Courier Status']=='Unshipped'].sort_values(by='months',ascending=True)
shipped=qty3[qty3['Courier Status']=='Shipped'].sort_values(by='months',ascending=True)
sns.catplot(data=unshipped,y='count',x='Category',kind='bar',hue='months',col=1,hue_order=mes)
plt.xticks(rotation=100)
plt.title('Unshipped')
unshipped
sns.catplot(data=shipped,y='count',x='Category',kind='bar',hue='months',col=1,hue_order=mes)
plt.xticks(rotation=90)
plt.title('Shipped')
shipped


Scopriamo che la maggior parte dei clienti ha acquistato 1 solo capo di abbigliamento, mentre pochi 2 e ancora meno 3.

- La maggior parte dei clienti ha acquistato ad Aprile,Maggio e Giugno 1 quantità.

-C'è però un'incogruenza perchè abbiamo 5091 persone che hanno acquistato 0 capi.
Due clienti hanno acquistato ben 5 volte 0 capi e 46 clienti 3 volte 0 capi, 260 2 volte e 4378 1 volta.
Quindi la maggior parte degli stessi clienti ha acquistato 1 volta 0 capi (quindi man mano che dimuisce la quantità, aumentano il numero di clienti che acquistano 0 capi)
Pochissimi a marzo,1958 ad aprile,1540 a maggio e 1190 a giugno (quindi il maggior numero si è avuto tra aprile e maggio).
Ma com'è possibile?

- La kurta è il capo più non acquistato (1953) seguito da il set.

- Scopriamo però che la maggior parte dei capi è stato acquistato a 0 e solo (kurta e set, quindi c'è un problema li e ci potrebbe essere anche una correlazione con i valori anomali che abbassano il guadagno) e di più ad Aprile.
Che cosa è successo ai prezzi di questi articoli nel mese di Aprile?
A maggio anche se meno che ad Aprile mentre a Giugno soltanto i set ma ancora di meno.
E scopriamo anche che queste quantità 0 non sono state acquistate con nessuna promozione.

- Tutte le persone che hanno acquistato a 0 (5091) sono in realtà ordini cancellati e quindi anche i kurta e i set sono dovuti a questo (4,33%).

- Ma capiamo anche che nella quantità 1 abbiamo 42037 kurta spediti e 2546 non spediti (il 5,7% non spedito e il 94,28% spedito, quindi una minima parte). Quindi gli ordini cancellati non si verificano soltanto nella quantità 0.
Lo stesso per quanto riguarda i set (spediti 39216 e non spediti 2387, quindi 5,7% non spediti e 94,26% spediti).ma anche gli altri articoli seppur di meno sono stati non spediti. Perchè questa mancata spedizione?
Questo si verifica anche nella categoria 2 anche se di meno (es. kurta spediti 92 % e non spediti 7,91%) mentre nella categoria 3 no.

- Per quanto riguarda i kurta sono non spediti a Giugno 1079, 843 ad aprile contro i 624 di Maggio mentre a Marzo 0 (quantità 1). I set 934 Giugno, 835 aprile, 615 maggio e 3 marzo.
Pochi nella quantità 2 e 0 nella quantità 3.

- Dal grafico vediamo chiaramente vediamo che sia per i kurta che per i set ad aprile hanno avuto lo stesso numero di ordini non spediti alto mentre a maggio vi è stata una decrescita e a giugno una risalita più alta rispetto ad aprile soprattutto per i set mentre per gli ordini spediti nei set e i kurta notiamo una decrescita graduale tra aprile e giugno ma anche per gli altri articoli.
Vediamo infatti il guadagno decrescere dal grafico.


In [ ]:
new_df['Amount']
xticks=['March','April','May','June']
sns.catplot(data=guadagno,x='months',y='Amount',order=xticks,kind='bar')
plt.show()

In [ ]:

guadagno
guadagno_dettagliato=new_df[['Amount','months','Category']]#[new_df['Category']!='kurta']
guadagno_dettagliato

In [ ]:
print(df['Status'].value_counts())
resi=df['Status'][(df['Status']=='Shipped - Returned to Seller')|(df['Status']=='Shipped - Returning to Seller')]
persi=df['Status'][df['Status']=='Shipped - Lost in Transit']
cancellati=df['Status'][df['Status']=='Cancelled']
sospesi=df['Status'][(df['Status']=='Pending')|(df['Status']=='Pending - Waiting for Pick Up')]
resi,cancellati,sospesi,persi
new_df['resi']=resi
new_df['persi']=persi
new_df['cancellati']=cancellati

Quindi c'è una correlazione decrescente tra il guadagno dettagliato per mesi di tutti gli articoli tranne i kurta e il guadagno dei kurta di -0,38. Ciò vuol dire che è possibile utilizzare la regressione lineare.
Abbiamo una correlazione crescente di 0.71 tra set e guadagno e ciò vuol dire che i set al contrario dei kurta fanno aumentare il guadagno mentre abbiamo una correlazione negativa di -0.54 tra kurta e set ciò vuol dire che come normalmente è all'aumentaree del guadagno dell'uno diminuisce l'altro.
Notiamo che la correlazione positiva scende se si considera il rapporto tra il guadagno e altri articoli come il western dress di 0.25 e con set e kurta di -0.26 quindi incidono di meno l'uno sul guadagno dell'altro.
E cala ancora di più con top arrivando ai -0.0610 e con gli altri 0.22.
Quindi la concentrazione è sui i set e i kurta.
La correlazione in generale tra guadagni mensili e guadagni dei kurta è -0.42.
Notiamo comunque che c'è una correlazione negativa intorno ai -0,27 tra western dress, kurta e set.


Scarsamente correlato il guadagno con stato di spedizione e promozioni,anche se con le promozioni abbiamo lo 0,34.
Notiamo però che se le promozioni incidono positivamente sul guadagno dei western dress ma negativamente sui kurta e i set con correlazione negativa dello -0,042 e -0,024.
La quantità però incide sullo stato di spedizione e viceversa.
C'è però una correlazione decrescente tra tipo di spedizione e promozioni di -0,69.

La stessa decrescita si osserva nei kurta e nei set. Anche se sappiamo che mediamente per i kurta si è speso a Marzo 462$, aprile 432$, a maggio 473$, a giugno $465. Ciò vuol dire che aumenta e decresce costantemente. Notiamo però man mano che passano i mesi il suo prezzo massimo si sta avvicinando al prezzo massimo in generale di 1296$.
Abbiamo una correlazione negativa tra il guadagno mensile e il guadagno mensile della kurta di -4,42,quindi considerevole.
Al decrescere dell'uno, abbiamo l'aumentare dell'altro.
Potremmo usare la regressione lineare per prevedere il prossimo mese, Luglio, quanto aumenterà il guadagno o dimuinirà in base alle future vendite dei kurta.

Ipotesi

- cannabilizzazione dei kurta nei confronti degli altri prodotti.
- impatto positivo dei set sul guadagno.
- scarso impatto della quantità ma anche degli altri fattori come lo stato del corriere e promozioni sul guadagno (tranne promozioni impatto positivo).
- effetto negativo delle promozioni sui kurta e sui set.
- correlazione positiva tra stato corriere e quantità.
- correlazione negativa tra tipo di spedizione e promozione.
- impatto negativo degli ordini gratis sul guadagno di -0,36 e sui kurta e i set del -0,10.

Obiettivi previsione

 - prevedere l'impatto e il numero futuro di ordini 0.0 sul guadagno utilizzando anche i dati in base alla stagionalità.
 - prevedere il guadagno prossimo di Luglio in base al guadagno dei kurta e quanto incide negativamente.
 - prevedere la relazione tra i kurta e gli altri prodotti e quanto aumenterà il guadagno
 - prevedere il numero di resi o ordini persi nel transito.

# Data preprocessing

Valori nulli

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
#new_df.drop(columns=['Unnamed: 22','outliers'],inplace=True)
new_df
new_df.isnull()[new_df.isnull()==True].sum()


Creazione altro dataset

In [ ]:
#outliers=df['Amount'].iloc[indice]
#new_df['outliers']=outliers

df_train=new_df[['Status','Courier Status','promotion-ids','ship-service-level','Category','Qty','Amount','fulfilled-by']]
#freeordermanca
#df_train.dropna()
df_train




In [ ]:
amountt=np.array(df_train['Amount']).reshape(-1,1)
imp_cat=SimpleImputer(strategy='median').set_output(transform='pandas')
amount_median=imp_cat.fit_transform(amountt)
df_train['Amount_median']=amount_median
df_train[df_train['Amount_median']==df_train['Amount'].median()]
df_train

sns.histplot(df_train['Amount_median'])

imp_mode1=SimpleImputer(strategy='most_frequent').set_output(transform='pandas')
imp_mode2=SimpleImputer(strategy='most_frequent').set_output(transform='pandas')
imp_mode3=SimpleImputer(strategy='most_frequent').set_output(transform='pandas')

transform=make_column_transformer((imp_mode1,['Courier Status']),
                                  (imp_mode2,['promotion-ids']),
                                  (imp_mode3,['fulfilled-by']))
df1=transform.fit_transform(df_train)
df2=pd.DataFrame(df1,columns=['Courier Status2','promotion-ids1','fulfilled-by1'])
unione=df_train.join(df2,how='inner')
unione=unione.drop(columns=['Amount','Courier Status','promotion-ids','fulfilled-by'])
dati_previsione=unione.copy()
dati_previsione

In [ ]:
encoder=OneHotEncoder(handle_unknown='ignore',sparse_output=False).set_output(transform='pandas')
colonne=unione[['Category','promotion-ids1','fulfilled-by1']]
#colonne1=np.array(colonne).reshape(-1,1)
encoder1=encoder.fit_transform(colonne)
encoder1
encoder_ordinal=OrdinalEncoder().set_output(transform='pandas')
ordinal=encoder_ordinal.fit_transform(unione[['Status','ship-service-level','Courier Status2']])
unione[['Status','ship-service-level','Courier Status2']]=ordinal
tutti_dati=unione.join(encoder1,how='inner')
tutti_dati.drop(columns=['Category','promotion-ids1','fulfilled-by1'])
tutti_dati

In [ ]:
X=tutti_dati[tutti_dati.columns[tutti_dati.columns!='Amount_median']]
y=tutti_dati['Amount_median']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=False)
regr=LinearRegression()
regr.fit(X_train,y_train)
y_new=regr.predict(X_test)
y_new

In [ ]:
anomalie=[]
months_1=[]
for m in mes:
    lista2=new_df[['Category','Amount','months']][(new_df['Category']=='kurta')&(new_df['months']==m)]['Amount'].sum()
    anomalie.append(lista2)
    kurta=pd.DataFrame(anomalie,columns=['guadagno'])
    months_1.append(m)
    kurta['months']=months_1


corr=guadagno['Amount'].corr(kurta['guadagno'])
kurta[['months_2','kurta']]=guadagno[['months','Amount']]
def spostamento(df,column1,column2):
   copy=df.copy()
   df['kurta'].iloc[column1]=copy['kurta'].iloc[column2]

spostamento(kurta,0,2)
spostamento(kurta,1,0)
spostamento(kurta,2,3)
spostamento(kurta,1,3)
kurta.drop(['months_2'],inplace=True,axis=1)
print(corr)

kurta_totale=new_df[['Amount','months','Category']][new_df['Category']=='kurta']
set_totale=new_df[['Amount','months','Category']][new_df['Category']=='Set']
western_dress=new_df[['Amount','months','Category']][new_df['Category']=='Western Dress']
kurta_totale.rename(columns={'Amount':'kurta','months':'months_1'},inplace=True)
set_totale.rename(columns={'Amount':'set','months':'months_2'},inplace=True)
western_dress.rename(columns={'Amount':'western dress','months':'months_3'},inplace=True)#.reset_index(drop=True,inplace=True)
data=pd.concat([guadagno_dettagliato,kurta_totale,set_totale,western_dress],axis=1)
data=data.fillna(0)
data['Qty']=new_df['Qty']
#new_df['Status']=pd.factorize(new_df['Status'])[0]
#new_df['Courier Status']=pd.factorize(new_df['Courier Status'])[0]
#new_df['promotion-ids']=pd.factorize(new_df['promotion-ids'])[0]
#new_df['ship-service-level']=pd.factorize(new_df['ship-service-level'])[0]
#new_df['free_order']=pd.factorize(new_df['free_order'])[0]
#data[['Status','Courier Status','promotion-ids','ship-service-level']]=new_df[['Status','Courier Status','promotion-ids','ship-service-level']]
#data['free_order']=new_df['free_order']
#data['outliers']=new_df['outliers']
#new_df['resi']=pd.factorize(new_df['resi'])[0]
#new_df['persi']=pd.factorize(new_df['persi'])[0]
#new_df['cancellati']=pd.factorize(new_df['cancellati'])[0]
#data['resi']=new_df['resi']
#data['persi']=new_df['persi']
#data['cancellati']=new_df['cancellati']

#corr_1=data.corr(numeric_only=True)
data

#sns.scatterplot(data=data,x='Amount',y='Amount_1',hue='months')
#sns.scatterplot(data=data,x='Amount',y='kurta',hue='months')
plt.show()
#corr_1